### Wine rating classification

In [1]:
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

In [2]:
mpt = MyPyTable()
filename = "input_data/WineRatings.csv"
mpt.load_from_file(filename)
mpt.column_names[0] = 'Name'

winestyle = mpt.get_column("Winestyle")
winestyles = myutils.get_unique_labels(winestyle)
print(winestyles)

['Red', 'Rose', 'White', 'Sparkling']


Investigate missing values, missing values in the dataset are denoted as "N.V."

In [3]:
# count the missing values in the dataset
count = myutils.count_nv(mpt.data)
print(f'There are {count} rows with missing data')

mpt.replace_missing_values_with_column_average('Price')
mpt.replace_missing_values_with_column_average('Rating')
mpt.replace_missing_values_with_column_average('Year')
summary_table = mpt.compute_summary_statistics(["Price", "Rating", "Year"])
summary_table.pretty_print()

# count the missing values in the dataset
count = myutils.count_nv(mpt.data)
print(f'There are {count} rows with missing data')

There are 744 rows with missing data
attribute        min      max      mid         avg    median
-----------  -------  -------  -------  ----------  --------
Price           3.15  3410.79  1706.97    33.0249      15.95
Rating          2.2      4.9      3.55     3.86566      3.9
Year         1961     2020     1990.5   2015.53      2016
There are 0 rows with missing data
